In [ ]:
import compyute as cp
import numpy as np
import torch

In [ ]:
B, T, C, H = 1, 5, 8, 4
x = cp.random.uniform((B, T, C), dtype="float32")
x_torch = torch.tensor(x.to_numpy(), requires_grad=True)

In [ ]:
from transformer import MultiHeadAttention

mha = MultiHeadAttention(in_channels=C, n_heads=H, bias=False)

In [ ]:
mha_torch = torch.nn.MultiheadAttention(C, H, bias=False, batch_first=True)

# PyTorch implements MHA as a batched matrix multiplication, they therefore only have one input proj matrix containing queries, keys and values for all heads
in_proj_weights = [
    # queries of head 1-4
    mha.modules[0].query.w,
    mha.modules[1].query.w,
    mha.modules[2].query.w,
    mha.modules[3].query.w,
    
    # keys of head 1-4
    mha.modules[0].key.w,
    mha.modules[1].key.w,
    mha.modules[2].key.w,
    mha.modules[3].key.w,

    # values of head 1-4
    mha.modules[0].value.w,
    mha.modules[1].value.w,
    mha.modules[2].value.w,
    mha.modules[3].value.w,
]
in_proj_weights = cp.concatenate(in_proj_weights, axis=0)
out_proj_weights = mha.modules[-1].w

mha_torch.in_proj_weight = torch.nn.Parameter(torch.tensor(in_proj_weights.to_numpy(), requires_grad=True))
mha_torch.out_proj.weight = torch.nn.Parameter(torch.tensor(out_proj_weights.to_numpy(), requires_grad=True))

In [ ]:
print(in_proj_weights.shape)
print(out_proj_weights.shape)

In [ ]:
print(mha_torch.in_proj_weight.shape)
print(mha_torch.out_proj.weight.shape)

In [ ]:
with mha.training():
    out = mha(x)
out_torch, _ = mha_torch(x_torch, x_torch, x_torch, need_weights=False)

In [ ]:
tol = 1e-6

In [ ]:
np.allclose(out.to_numpy(), out_torch.detach().numpy(), atol=tol, rtol=tol)

In [ ]:
dy = cp.random.normal(out.shape)
with mha.training():
    dx = mha.backward(dy)
out_torch.backward(torch.tensor(dy.to_numpy()))

np.allclose(dx.to_numpy(), x_torch.grad.detach().numpy(), atol=tol, rtol=tol)

In [ ]:
in_proj_weight_grads = [
    # query grads of head 1-4
    mha.modules[0].query.w.grad,
    mha.modules[1].query.w.grad,
    mha.modules[2].query.w.grad,
    mha.modules[3].query.w.grad,

    # key grads of head 1-4
    mha.modules[0].key.w.grad,
    mha.modules[1].key.w.grad,
    mha.modules[2].key.w.grad,
    mha.modules[3].key.w.grad,

    # value grads of head 1-4
    mha.modules[0].value.w.grad,
    mha.modules[1].value.w.grad,
    mha.modules[2].value.w.grad,
    mha.modules[3].value.w.grad,
]
in_proj_weight_grads = cp.concatenate(in_proj_weight_grads, axis=0)
out_proj_weight_grads = mha.modules[-1].w.grad

In [ ]:
np.allclose(in_proj_weight_grads.to_numpy(), mha_torch.in_proj_weight.grad.detach().numpy(), atol=tol, rtol=tol)

In [ ]:
np.allclose(out_proj_weight_grads.to_numpy(), mha_torch.out_proj.weight.grad.detach().numpy(), atol=tol, rtol=tol)